In [2]:
import pandas as pd 
import numpy as np
import spacy 
import random
from spacy.util import minibatch, compounding
from pathlib import Path
import matplotlib.pyplot as plt
import re
#from wordcloud import WordCloud, STOPWORDS
from spacy.util import minibatch, compounding

In [67]:
#load data 
df = pd.read_csv("FOOD list1.csv")

In [71]:
df.head(10)

,Food Name
0,Jelly
1,Honey
2,Butter
3,Eggs
4,Banana
5,Cake
6,Firnee
7,Halua
8,Tea
9,Coffee


In [72]:
nlp = spacy.load('en_core_web_sm')

In [73]:
nlp

In [74]:
# Get All Components of this NLP Object
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [75]:
ner = nlp.get_pipe('ner')

In [76]:
# Example
ex1 = "MR. Jawad eat kacchi biriyani every days"
ex2 = "James went to London to buy Ibuprofen last year 2019"

doc = nlp(ex2)
# Check for entities
for entity in doc.ents:
  print(entity,entity.label_)

London GPE
Ibuprofen ORG
last year 2019 DATE


In [77]:
#preparing the data 

def food_name(name):
    processed_token = []
    for token in name.split():
        token = ''.join(e.lower() for e in token if e.isalnum())
        processed_token.append(token)
    return ' '.join(processed_token)

In [96]:
# Food Names
df.head()
all_food = df['Food Name'].unique().tolist()

In [79]:
all_food = [x.lower() for x in all_food]

In [82]:
#all_food

In [83]:
count = 0
TRAIN_DATA = []
for _, item in df.iterrows():
    ent_dict = {}
    if count < 100000:
        name = food_name(item['Food Name'])
        #Locate foods and their positions
        visited_items = []
        entities = []
        for token in name.split():
            if token in all_food:
                for i in re.finditer(token, name):
                        entity = (i.span()[0], i.span()[1], 'FOOD')
                        visited_items.append(token)
                        entities.append(entity)
        if len(entities) > 0:
            ent_dict['entities'] = entities
            train_item = (name, ent_dict)
            TRAIN_DATA.append(train_item)
            count+=1

In [84]:
#all_food

In [85]:
TRAIN_DATA

[('jelly', {'entities': [(0, 5, 'FOOD')]}),
 ('honey', {'entities': [(0, 5, 'FOOD')]}),
 ('butter', {'entities': [(0, 6, 'FOOD')]}),
 ('eggs', {'entities': [(0, 4, 'FOOD')]}),
 ('banana', {'entities': [(0, 6, 'FOOD')]}),
 ('cake', {'entities': [(0, 4, 'FOOD')]}),
 ('firnee', {'entities': [(0, 6, 'FOOD')]}),
 ('halua', {'entities': [(0, 5, 'FOOD')]}),
 ('tea', {'entities': [(0, 3, 'FOOD')]}),
 ('coffee', {'entities': [(0, 6, 'FOOD')]}),
 ('parata', {'entities': [(0, 6, 'FOOD')]}),
 ('chapati', {'entities': [(0, 7, 'FOOD')]}),
 ('toast', {'entities': [(0, 5, 'FOOD')]}),
 ('sugar', {'entities': [(0, 5, 'FOOD')]}),
 ('milk', {'entities': [(0, 4, 'FOOD')]}),
 ('bhajee', {'entities': [(0, 6, 'FOOD')]}),
 ('banana', {'entities': [(0, 6, 'FOOD')]}),
 ('papaya', {'entities': [(0, 6, 'FOOD')]}),
 ('apple', {'entities': [(0, 5, 'FOOD')]}),
 ('orange', {'entities': [(0, 6, 'FOOD')]}),
 ('grapes', {'entities': [(0, 6, 'FOOD')]}),
 ('mango', {'entities': [(0, 5, 'FOOD')]}),
 ('fish', {'entities': [(

###### Training the Food NER model

In [86]:
# Getting the ner component
ner=nlp.get_pipe('ner')

In [97]:
n_iter = 10
def train_ner(training_data):
    """Steps
    Create a Blank NLP  model object
    Create and add NER to the NLP model
    Add Labels from your training data
    Train  
    """
    TRAIN_DATA = training_data
    nlp = spacy.blank("en")  # create blank Language class
    print("Created blank 'en' model")
    
    if "ner" not in nlp.pipe_names:
        ner =  nlp.add_pipe("ner")
        #nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
        
    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            
    nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            example = []
            # Update the model with iterating each text
            from spacy.training import Example
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
                #update the model
            ner.update(
                example,  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                losses=losses,
            )
        print("Losses", losses)
    return nlp

In [98]:
nlp=train_ner(TRAIN_DATA)

Created blank 'en' model
Losses {'ner': 39.16666841506958}
Losses {'ner': 39.16666841506958}
Losses {'ner': 39.16666841506958}
Losses {'ner': 39.16666841506958}
Losses {'ner': 39.16666841506958}
Losses {'ner': 39.16666841506958}
Losses {'ner': 39.16666841506958}
Losses {'ner': 39.16666841506958}
Losses {'ner': 39.16666841506958}
Losses {'ner': 39.16666841506958}


In [99]:
# Test
for text,_ in TRAIN_DATA[:1000]:
    doc = nlp1(text)
    result = [(ent,ent.label_) for ent in doc.ents]
    print(result)

[(graps, 'FOOD')]
[(vharta, 'FOOD')]
[(sandwitch, 'FOOD')]
[(mango, 'FOOD')]
[(salad, 'FOOD')]
[(bhajee, 'FOOD')]
[(korolla, 'FOOD')]
[(halua, 'FOOD')]
[(banana, 'FOOD')]
[(burger, 'FOOD')]
[(muri, 'FOOD')]
[(mango, 'FOOD')]
[(bhajee, 'FOOD')]
[(grapes, 'FOOD')]
[(parata, 'FOOD')]
[(sugar, 'FOOD')]
[(shutki, 'FOOD')]
[(coffee, 'FOOD')]
[(jelly, 'FOOD')]
[(orange, 'FOOD')]
[(toast, 'FOOD')]
[(cake, 'FOOD')]
[(honey, 'FOOD')]
[(butter, 'FOOD')]
[(banana, 'FOOD')]
[(potol, 'FOOD')]
[(lobster, 'FOOD')]
[(eggs, 'FOOD')]
[(tea, 'FOOD')]
[(papaya, 'FOOD')]
[(doi, 'FOOD')]
[(firnee, 'FOOD')]
[(frenchfry, 'FOOD')]
[(bharta, 'FOOD')]
[(milk, 'FOOD')]
[(fish, 'FOOD')]
[(goza, 'FOOD')]
[(apple, 'FOOD')]
[(chapati, 'FOOD')]
[(mishti, 'FOOD')]
[(prawn, 'FOOD')]
[(mineral, 'FOOD')]
[(muffin, 'FOOD')]
[(jackfruite, 'FOOD')]
[(chira, 'FOOD')]
[(kensington, 'FOOD')]
[(hotdog, 'FOOD')]


In [100]:
def extract_food_entity(text):
    docx =  nlp1(text)
    result = [(ent,ent.label_) for ent in docx.ents]
    return result

In [101]:
result

[(hotdog, 'FOOD')]

In [102]:
df['Food Name'][0:10]

0     Jelly
1     Honey
2    Butter
3      Eggs
4    Banana
5      Cake
6    Firnee
7     Halua
8       Tea
9    Coffee
Name: Food Name, dtype: object

In [103]:
df['Food Name'][0:10].apply(extract_food_entity)

0     [((Jelly), FOOD)]
1     [((Honey), FOOD)]
2    [((Butter), FOOD)]
3      [((Eggs), FOOD)]
4    [((Banana), FOOD)]
5      [((Cake), FOOD)]
6    [((Firnee), FOOD)]
7     [((Halua), FOOD)]
8       [((Tea), FOOD)]
9    [((Coffee), FOOD)]
Name: Food Name, dtype: object

In [104]:
ex3 = "Rahim's favourite fish bhoona cooked by his sister . His mother cooked Firnee for him"

In [105]:
doc1 = nlp1(ex3)
for entity in doc1.ents:
    print(entity,entity.label_)

Rahim FOOD
's FOOD
favourite FOOD
fish FOOD
bhoona FOOD
cooked FOOD
by FOOD
his FOOD
sister FOOD
. FOOD
His FOOD
mother FOOD
cooked FOOD
Firnee FOOD
for FOOD
him FOOD
